# 전기차 등록수 data 행정동 단위 에서 격자단위로 변환
전기차 등록 대수 행정동 단위로 격자위에 뿌려짐(evenly) -> 행정동전기차등록수 * 격자건물수/행정동건물수  격자단위 적용

In [6]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm 

In [10]:
df_build= gpd.read_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/(B100)국토통계_건축물-건축물 수 합계 통계-(격자) 100M_경기도 용인시_202205/nlsp_021002021.shp',encoding='utf-8')
df_grid=gpd.read_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered_v3(ev_cnt).geojson')
df_build.fillna(0,inplace=True)

In [11]:
df_build.head()

,gid,lbl,val,geometry
0,다사748072,1.00,1.0,"POLYGON ((974800.000 1907200.000, 974800.000 1..."
1,다사750101,2.00,2.0,"POLYGON ((975000.000 1910100.000, 975000.000 1..."
2,다사914045,2.00,2.0,"POLYGON ((991400.000 1904500.000, 991400.000 1..."
3,다사918030,7.00,7.0,"POLYGON ((991800.000 1903000.000, 991800.000 1..."
4,다사708265,3.00,3.0,"POLYGON ((970800.000 1926500.000, 970800.000 1..."


In [12]:
df_grid.head()

,gid,lbl,val,longitude,latitude,읍면동,ev_cnt,geometry
0,다사750101,N/A,0.0,127.218888,37.189802,이동읍,98.0,POINT (975240.056 1909843.864)
1,다사914045,N/A,0.0,127.403730,37.139617,백암면,67.0,POINT (991640.264 1904244.063)
2,다사918030,N/A,0.0,127.408251,37.126099,백암면,67.0,POINT (992040.289 1902744.057)
3,다사595284,N/A,0.0,127.043257,37.354210,동천동,348.0,POINT (959739.665 1928143.776)
4,다사720059,N/A,0.0,127.185244,37.151858,남사읍,173.0,POINT (972240.096 1905643.788)


In [13]:
df_build=df_build.drop(columns=['lbl','geometry'])
df_build.rename(columns = {'val' : 'grid_bld_cnt'}, inplace = True)
result=df_grid.merge(df_build,on=['gid'])

In [14]:
dic=result.groupby('읍면동')['grid_bld_cnt'].sum().items()
dic = dict(dic)

In [15]:
result['동별건물cnt']=result['읍면동']
result.replace({'동별건물cnt':dic},inplace=True)

In [16]:
result['동별건물cnt'].value_counts()

1997.0    2006
2313.0    1681
2212.0    1346
2660.0    1274
1264.0    1244
2118.0    1195
778.0      998
888.0      512
299.0      420
425.0      378
643.0      353
511.0      324
347.0      304
491.0      256
467.0      182
214.0      172
148.0      168
326.0      130
0.0        130
246.0       94
232.0       71
24.0        62
37.0        53
164.0       51
85.0        33
1.0          7
Name: 동별건물cnt, dtype: int64

In [20]:
tqdm.pandas()
result['ev_cnt'] = result.progress_apply(lambda row : row.ev_cnt*row.grid_bld_cnt/(row.동별건물cnt+1),axis=1)

100%|██████████| 14288/14288 [00:00<00:00, 31377.16it/s]


In [24]:
result.drop(columns=['lbl','longitude','latitude','동별건물cnt'],inplace=True)

In [26]:
result.to_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered_v4(ev_cnt by building_cnt).geojson',index=False,encoding='utf-8')